In [6]:
import pandas as pd
import numpy as np

import seaborn as sns
from matplotlib import pyplot as plt
%matplotlib inline

In [8]:

data = "https://raw.githubusercontent.com/alexeygrigorev/datasets/master/car_fuel_efficiency.csv"

!wget $data

290.41s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


--2025-11-07 02:04:13--  https://raw.githubusercontent.com/alexeygrigorev/datasets/master/car_fuel_efficiency.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 874188 (854K) [text/plain]
Saving to: ‘car_fuel_efficiency.csv.1’

car_fuel_efficiency 100%[===================>] 853.70K  --.-KB/s    in 0.05s   

2025-11-07 02:04:13 (16.6 MB/s) - ‘car_fuel_efficiency.csv.1’ saved [874188/874188]



In [9]:
!head car_fuel_efficiency.csv

369.17s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


engine_displacement,num_cylinders,horsepower,vehicle_weight,acceleration,model_year,origin,fuel_type,drivetrain,num_doors,fuel_efficiency_mpg
170,3,159,3413.433758606219,17.7,2003,Europe,Gasoline,All-wheel drive,0,13.231728906241411
130,5,97,3149.6649342200353,17.8,2007,USA,Gasoline,Front-wheel drive,0,13.688217435463793
170,,78,3079.03899736884,15.1,2018,Europe,Gasoline,Front-wheel drive,0,14.246340998160866
220,4,,2542.392401828378,20.2,2009,USA,Diesel,All-wheel drive,2,16.91273559598635
210,1,140,3460.870989989018,14.4,2009,Europe,Gasoline,All-wheel drive,2,12.488369121964562
190,3,,2484.883986036068,14.7,2008,Europe,Gasoline,All-wheel drive,-1,17.271818372724237
240,7,127,3006.5422872171457,22.2,2012,USA,Gasoline,Front-wheel drive,1,13.210412112385608
150,4,239,3638.6577802809,17.3,2020,USA,Diesel,All-wheel drive,1,12.848883861524026
250,1,174,2714.219309645285,10.3,2016,Asia,Diesel,Front-wheel drive,-1,16.823553726916543


In [28]:
df = pd.read_csv('/home/mirodov/ml-camp/homework/06/car_fuel_efficiency.csv')
df.head()


num_doors
 0.0    3551
 1.0    2192
-1.0    2183
-2.0     594
 2.0     563
 NaN     502
 3.0      58
-3.0      56
-4.0       4
 4.0       1
Name: count, dtype: int64

In [27]:
df.isnull().sum()

engine_displacement      0
num_cylinders          482
horsepower             708
vehicle_weight           0
acceleration           930
model_year               0
origin                   0
fuel_type                0
drivetrain               0
num_doors              502
fuel_efficiency_mpg      0
dtype: int64

In [31]:
#Fill missing values with zeros.
df = df.fillna(0)
df.isnull().sum()

engine_displacement    0
num_cylinders          0
horsepower             0
vehicle_weight         0
acceleration           0
model_year             0
origin                 0
fuel_type              0
drivetrain             0
num_doors              0
fuel_efficiency_mpg    0
dtype: int64

In [32]:
df['num_doors'].value_counts(dropna=False)

num_doors
 0.0    4053
 1.0    2192
-1.0    2183
-2.0     594
 2.0     563
 3.0      58
-3.0      56
-4.0       4
 4.0       1
Name: count, dtype: int64

In [33]:
#Do train/validation/test split with 60%/20%/20% distribution.
# Use the train_test_split function and set the random_state parameter to 1.
from sklearn.model_selection import train_test_split

df_full_train, df_test = train_test_split(df, test_size=0.2, random_state=1)
df_train, df_val = train_test_split(df_full_train, test_size=0.25, random_state=1)

df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

y_train = (df_train.fuel_efficiency_mpg).values
y_val = (df_val.fuel_efficiency_mpg).values
y_test = (df_test.fuel_efficiency_mpg).values

del df_train['fuel_efficiency_mpg']
del df_val['fuel_efficiency_mpg']
del df_test['fuel_efficiency_mpg']


In [ ]:
df_train

In [47]:
# Use DictVectorizer(sparse=True) to turn the dataframes into matrices.
from sklearn.feature_extraction import DictVectorizer
dv = DictVectorizer(sparse=True)
train_dicts = df_train.fillna(0).to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)


In [ ]:
train_dicts
X_train
dv.get_feature_names_out()

In [49]:
#Question 1
"""
Let's train a decision tree regressor to predict the fuel_efficiency_mpg variable.

Train a model with max_depth=1.
Which feature is used for splitting the data?

'vehicle_weight'
'model_year'
'origin'
'fuel_type'
"""
from sklearn.tree import DecisionTreeRegressor
dt = DecisionTreeRegressor(max_depth=1)
dt.fit(X_train, y_train)

DecisionTreeRegressor(max_depth=1)

In [52]:
from sklearn.tree import export_text
print(export_text(dt, feature_names=list(dv.get_feature_names_out())))

|--- vehicle_weight <= 3022.11
|   |--- value: [16.88]
|--- vehicle_weight >  3022.11
|   |--- value: [12.94]



In [ ]:
"""
Question 2
Train a random forest regressor with these parameters:

n_estimators=10
random_state=1
n_jobs=-1 (optional - to make training faster)
What's the RMSE of this model on the validation data?

0.045
0.45
4.5
45.0
"""

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import root_mean_squared_error

rf = RandomForestRegressor(n_estimators=10, random_state=1, n_jobs=-1)
rf.fit(X_train, y_train)
val_dicts = df_val.fillna(0).to_dict(orient='records')
X_val = dv.transform(val_dicts)
y_pred = rf.predict(X_val)

rmse = root_mean_squared_error(y_val, y_pred)
print(rmse)


0.45957772230927263


In [58]:
"""
Question 3
Now let's experiment with the n_estimators parameter

Try different values of this parameter from 10 to 200 with step 10.
Set random_state to 1.
Evaluate the model on the validation dataset.
After which value of n_estimators does RMSE stop improving? Consider 3 decimal places for calculating the answer.

10
25
80
200
If it doesn't stop improving, use the latest iteration number in your answer.
"""

n_estimators_options = list(range(10, 201, 10))
best_rmse = float("inf")
best_n_estimators = 0
for n_estimators in n_estimators_options:
    rf = RandomForestRegressor(n_estimators=n_estimators, random_state=1, n_jobs=-1)
    rf.fit(X_train, y_train)
    y_pred = rf.predict(X_val)
    rmse = root_mean_squared_error(y_val, y_pred)
    print(f"n_estimators: {n_estimators}, RMSE: {rmse}")
    if rmse < best_rmse:
        best_rmse = rmse
        best_n_estimators = n_estimators

n_estimators: 10, RMSE: 0.4595777223092726
n_estimators: 20, RMSE: 0.45359067251247054
n_estimators: 30, RMSE: 0.4516867257545712
n_estimators: 40, RMSE: 0.44872083017369974
n_estimators: 50, RMSE: 0.44665689724160934
n_estimators: 60, RMSE: 0.445459702608112
n_estimators: 70, RMSE: 0.44512632449869965
n_estimators: 80, RMSE: 0.44498431197772836
n_estimators: 90, RMSE: 0.4448614906399875
n_estimators: 100, RMSE: 0.4446518680868041
n_estimators: 110, RMSE: 0.4435787643986023
n_estimators: 120, RMSE: 0.44391186812338157
n_estimators: 130, RMSE: 0.443702590396687
n_estimators: 140, RMSE: 0.44335499551016877
n_estimators: 150, RMSE: 0.44289761494219454
n_estimators: 160, RMSE: 0.4427612219659299
n_estimators: 170, RMSE: 0.44280146504730894
n_estimators: 180, RMSE: 0.4423619535704136
n_estimators: 190, RMSE: 0.4424939711220691
n_estimators: 200, RMSE: 0.4424785084688597


In [60]:
"""
Question 4
Let's select the best max_depth:

Try different values of max_depth: [10, 15, 20, 25]
For each of these values,
try different values of n_estimators from 10 till 200 (with step 10)
calculate the mean RMSE
Fix the random seed: random_state=1
What's the best max_depth, using the mean RMSE?

10
15
20
25
"""
max_depth_options = [10, 15, 20, 25]
n_estimators_options = list(range(10, 201, 10))

best_overall_rmse = float("inf")
best_overall_max_depth = 0
for max_depth in max_depth_options:
    for n_estimators in n_estimators_options:
        rf = RandomForestRegressor(n_estimators=n_estimators, max_depth=max_depth, random_state=1, n_jobs=-1)
        rf.fit(X_train, y_train)
        y_pred = rf.predict(X_val)
        rmse = root_mean_squared_error(y_val, y_pred)
        print(f"max_depth: {max_depth}, n_estimators: {n_estimators}, RMSE: {rmse}")
        if rmse < best_overall_rmse:
            best_overall_rmse = rmse
            best_overall_max_depth = max_depth

max_depth: 10, n_estimators: 10, RMSE: 0.4502486597058524
max_depth: 10, n_estimators: 20, RMSE: 0.44685703362920204
max_depth: 10, n_estimators: 30, RMSE: 0.44547396459413735
max_depth: 10, n_estimators: 40, RMSE: 0.44306731129625837
max_depth: 10, n_estimators: 50, RMSE: 0.4419566862179356
max_depth: 10, n_estimators: 60, RMSE: 0.4416730330613033
max_depth: 10, n_estimators: 70, RMSE: 0.4412975503694072
max_depth: 10, n_estimators: 80, RMSE: 0.44143523500728943
max_depth: 10, n_estimators: 90, RMSE: 0.4415215165581006
max_depth: 10, n_estimators: 100, RMSE: 0.4412169979071018
max_depth: 10, n_estimators: 110, RMSE: 0.4405262272478249
max_depth: 10, n_estimators: 120, RMSE: 0.4407083659646052
max_depth: 10, n_estimators: 130, RMSE: 0.4406295000948249
max_depth: 10, n_estimators: 140, RMSE: 0.4403394127734899
max_depth: 10, n_estimators: 150, RMSE: 0.4399427035517263
max_depth: 10, n_estimators: 160, RMSE: 0.43979740503833176
max_depth: 10, n_estimators: 170, RMSE: 0.44001743947445027


In [ ]:
"""
Question 5
We can extract feature importance information from tree-based models.

At each step of the decision tree learning algorithm, it finds the best split. When doing it, we can calculate "gain" - the reduction in impurity before and after the split. This gain is quite useful in understanding what are the important features for tree-based models.

In Scikit-Learn, tree-based models contain this information in the feature_importances_ field.

For this homework question, we'll find the most important feature:

Train the model with these parameters:
n_estimators=10,
max_depth=20,
random_state=1,
n_jobs=-1 (optional)
Get the feature importance information from this model
What's the most important feature (among these 4)?

vehicle_weight
horsepower
acceleration
engine_displacement
"""
rf = RandomForestRegressor(n_estimators=10, max_depth=20, random_state=1, n_jobs=-1)
rf.fit(X_train, y_train)
fi = rf.feature_importances_
feature_names = dv.get_feature_names_out()
feature_importances = list(zip(feature_names, fi))
feature_importances.sort(key=lambda x: x[1], reverse=True)
for feature, importance in feature_importances[:10]:
    print(f"Feature: {feature}, Importance: {importance}")



In [ ]:
"""
Question 6

Now let's train an XGBoost model! For this question, we'll tune the eta parameter:

Install XGBoost
Create DMatrix for train and validation
Create a watchlist
Train a model with these parameters for 100 rounds:
xgb_params = {
    'eta': 0.3,
    'max_depth': 6,
    'min_child_weight': 1,

    'objective': 'reg:squarederror',
    'nthread': 8,

    'seed': 1,
    'verbosity': 1,
}
Now change eta from 0.3 to 0.1.

Which eta leads to the best RMSE score on the validation dataset?

0.3
0.1
Both give equal value
"""

import xgboost as xgb
dtrain = xgb.DMatrix(X_train, label=y_train)
dval = xgb.DMatrix(X_val, label=y_val)
watchlist = [(dtrain, 'train'), (dval, 'val')]
xgb_params_1 = {
    'eta': 0.3,
    'max_depth': 6,
    'min_child_weight': 1,
    'objective': 'reg:squarederror',
    'nthread': 8,
    'seed': 1,
    'verbosity': 1,
}
model_1 = xgb.train(xgb_params_1, dtrain, num_boost_round=100, evals=watchlist)
y_pred_1 = model_1.predict(dval)
rmse_1 = root_mean_squared_error(y_val, y_pred_1)
print(f"RMSE with eta=0.3: {rmse_1}")


[0]	train-rmse:1.81393	val-rmse:1.85444
[1]	train-rmse:1.31919	val-rmse:1.35353
[2]	train-rmse:0.98120	val-rmse:1.01316


[3]	train-rmse:0.75443	val-rmse:0.78667
[4]	train-rmse:0.60680	val-rmse:0.64318
[5]	train-rmse:0.51381	val-rmse:0.55664
[6]	train-rmse:0.45470	val-rmse:0.50321
[7]	train-rmse:0.41881	val-rmse:0.47254
[8]	train-rmse:0.39534	val-rmse:0.45509
[9]	train-rmse:0.38038	val-rmse:0.44564
[10]	train-rmse:0.37115	val-rmse:0.43896
[11]	train-rmse:0.36361	val-rmse:0.43594
[12]	train-rmse:0.35850	val-rmse:0.43558
[13]	train-rmse:0.35365	val-rmse:0.43394
[14]	train-rmse:0.35025	val-rmse:0.43349
[15]	train-rmse:0.34666	val-rmse:0.43362
[16]	train-rmse:0.34459	val-rmse:0.43378
[17]	train-rmse:0.34128	val-rmse:0.43405
[18]	train-rmse:0.33822	val-rmse:0.43391
[19]	train-rmse:0.33709	val-rmse:0.43374
[20]	train-rmse:0.33553	val-rmse:0.43376
[21]	train-rmse:0.33243	val-rmse:0.43453
[22]	train-rmse:0.33031	val-rmse:0.43510
[23]	train-rmse:0.32815	val-rmse:0.43601
[24]	train-rmse:0.32670	val-rmse:0.43592
[25]	train-rmse:0.32268	val-rmse:0.43683
[26]	train-rmse:0.32085	val-rmse:0.43678
[27]	train-rmse:0.32035

In [ ]:
import xgboost as xgb
dtrain = xgb.DMatrix(X_train, label=y_train)
dval = xgb.DMatrix(X_val, label=y_val)
watchlist = [(dtrain, 'train'), (dval, 'val')]
xgb_params_1 = {
    'eta': 0.1,
    'max_depth': 6,
    'min_child_weight': 1,
    'objective': 'reg:squarederror',
    'nthread': 8,
    'seed': 1,
    'verbosity': 1,
}
model_1 = xgb.train(xgb_params_1, dtrain, num_boost_round=100, evals=watchlist)
y_pred_1 = model_1.predict(dval)
rmse_1 = root_mean_squared_error(y_val, y_pred_1)
print(f"RMSE with eta=0.1: {rmse_1}")


[0]	train-rmse:2.28944	val-rmse:2.34561
[1]	train-rmse:2.07396	val-rmse:2.12434
[2]	train-rmse:1.88066	val-rmse:1.92597
[3]	train-rmse:1.70730	val-rmse:1.74987
[4]	train-rmse:1.55163	val-rmse:1.59059
[5]	train-rmse:1.41247	val-rmse:1.44988
[6]	train-rmse:1.28796	val-rmse:1.32329
[7]	train-rmse:1.17660	val-rmse:1.20930
[8]	train-rmse:1.07736	val-rmse:1.10830
[9]	train-rmse:0.98883	val-rmse:1.02009
[10]	train-rmse:0.91008	val-rmse:0.94062
[11]	train-rmse:0.84030	val-rmse:0.87100
[12]	train-rmse:0.77874	val-rmse:0.80916
[13]	train-rmse:0.72417	val-rmse:0.75465
[14]	train-rmse:0.67626	val-rmse:0.70780
[15]	train-rmse:0.63402	val-rmse:0.66672
[16]	train-rmse:0.59690	val-rmse:0.63062
[17]	train-rmse:0.56447	val-rmse:0.60016
[18]	train-rmse:0.53619	val-rmse:0.57383
[19]	train-rmse:0.51138	val-rmse:0.55044
[20]	train-rmse:0.48983	val-rmse:0.53064
[21]	train-rmse:0.47135	val-rmse:0.51451
[22]	train-rmse:0.45501	val-rmse:0.49998
[23]	train-rmse:0.44120	val-rmse:0.48790
[24]	train-rmse:0.42929	va

[52]	train-rmse:0.33767	val-rmse:0.42453
[53]	train-rmse:0.33651	val-rmse:0.42459
[54]	train-rmse:0.33560	val-rmse:0.42448
[55]	train-rmse:0.33480	val-rmse:0.42449
[56]	train-rmse:0.33386	val-rmse:0.42426
[57]	train-rmse:0.33292	val-rmse:0.42429
[58]	train-rmse:0.33196	val-rmse:0.42438
[59]	train-rmse:0.33105	val-rmse:0.42450
[60]	train-rmse:0.33054	val-rmse:0.42456
[61]	train-rmse:0.32953	val-rmse:0.42444
[62]	train-rmse:0.32872	val-rmse:0.42454
[63]	train-rmse:0.32777	val-rmse:0.42461
[64]	train-rmse:0.32673	val-rmse:0.42479
[65]	train-rmse:0.32602	val-rmse:0.42493
[66]	train-rmse:0.32489	val-rmse:0.42520
[67]	train-rmse:0.32397	val-rmse:0.42525
[68]	train-rmse:0.32338	val-rmse:0.42518
[69]	train-rmse:0.32275	val-rmse:0.42513
[70]	train-rmse:0.32202	val-rmse:0.42503
[71]	train-rmse:0.32161	val-rmse:0.42515
[72]	train-rmse:0.32060	val-rmse:0.42496
[73]	train-rmse:0.31996	val-rmse:0.42517
[74]	train-rmse:0.31963	val-rmse:0.42513
[75]	train-rmse:0.31895	val-rmse:0.42526
[76]	train-rmse: